<a href="https://colab.research.google.com/github/pkuSapphire/CreditRiskManagement/blob/main/Project_2_Time_Series_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Time Series Model

By Bosen Li (bl3097), Wenyu Luo (wl2905), Edward Zhang (yz4756)

## Step 1

In this step, we found out that both the card or the real state charge-off rate is not stationary, when we use 5% p-value as the standard. However, for the credit card loan, the p-value for ADF test is really close to 5%. Thus, we decided to keep the orignal charge-off data and conduct follow regressions for both the cre charge-off rates as well as its first difference.

In [8]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

cre = pd.read_excel('CRE.xlsx')
cre.set_index('date',inplace=True)
card = pd.read_excel('card.xlsx')
card.set_index('date',inplace=True)

# calculate pct
cre['pct'] = 100 *  cre['chargeoffs']/cre['loans']
card['pct'] = 100 * card['chargeoffs']/card['loans']

# adf test for pct
print(f"The p-value of CRE chargeoffs percentage's Augmented Dickie Fuller test is \
{adfuller(cre['pct'])[1]:.3f}, \
indicating that this is not stationary.")
print(f"The p-value of card chargeoffs percentage'sAugmented Dickie Fuller test is \
{adfuller(card['pct'])[1]:.3f}, \
indicating that this is not stationary too.\n")

# create first difference
cre['pct_change'] = cre['pct'].diff()
card['pct_change'] = card['pct'].diff()
cre.dropna(inplace=True)
card.dropna(inplace=True)

# adf test for first difference
print('After taking the first difference,')
print(f"The p-value of CRE chargeoffs percentage's Augmented Dickie Fuller test is \
{adfuller(cre['pct_change'])[1]:.3f}, \
thus that this is stationary.")
print(f"The p-value of card chargeoffs percentage'sAugmented Dickie Fuller test is \
{adfuller(card['pct_change'])[1]:.3f}, \
thus that this is stationary too.")


The p-value of CRE chargeoffs percentage's Augmented Dickie Fuller test is 0.489, indicating that this is not stationary.
The p-value of card chargeoffs percentage'sAugmented Dickie Fuller test is 0.053, indicating that this is not stationary too.

After taking the first difference,
The p-value of CRE chargeoffs percentage's Augmented Dickie Fuller test is 0.013, thus that this is stationary.
The p-value of card chargeoffs percentage'sAugmented Dickie Fuller test is 0.002, thus that this is stationary too.


## Step 2

In this step, we first fetch the data we need and align them up.
- Data like unrate and gdp is reported on the first day of a month, but standing for the last month. So we move it one day frontier, which is more reasonable.
- All data are resampled to the last day of a quarter. Accumulated data like gdp should be at then end of the quarter. Other data like oil, interest rate are highly related to price. If we are considering a portafolio, doing any kinds of forcasting, we should use the last day data because means and medium means nothing for evaluating.
- We conduct adfuller test for each, and preserved the stationary factors, while making first difference of others, or to say, 'pct change', as suggested by professor.
- We merge using date index, sort of like a left hand join.
- `resample('QE')` simply means resample by quarter, using `'QE'` instead of `'Q'` to avoid annoying warnings from python.
- Most of the factors we are using is actually stationary, so we can use them directly, yet the oil price is definetly not stationary as we guessed, so we made a first difference of it.

In [9]:
import pandas_datareader.data as web

unrate = web.DataReader("UNRATE","fred",start='2000-01-01')
oil = web.DataReader("DCOILBRENTEU","fred",start='2000-01-01')
gdp = web.DataReader("GDP","fred",start='2000-01-01')
t10y2y = web.DataReader("T10Y2Y","fred",start='2000-01-01')
volatity = web.DataReader("VIXCLS","fred",start='2000-01-01')

# cleaning data
unrate.index = unrate.index-pd.DateOffset(days=1) # This will not cause overlap, because the data itself is one day lag
unrate = unrate.resample('QE').last()
gdp.index = gdp.index-pd.DateOffset(days=1) # This will not cause overlap, because the data itself is one day lag
# gdp = gdp.resample('QE').last() # gdp is Q data, do not need any other change
oil = oil.resample('QE').last()
t10y2y = t10y2y.resample('QE').last()
volatity = volatity.resample('QE').last()

econ = pd.concat([unrate,oil,gdp,t10y2y,volatity],axis=1)
econ['gdp_growth'] = econ['GDP'].pct_change(fill_method=None) # econ['gdp_growth'] = econ['GDP'].diff()/econ['GDP'].shift(1), thank new features
econ.dropna(inplace=True)

# print(econ.head())

for col in ['UNRATE', 'DCOILBRENTEU', 'GDP', 'gdp_growth', 'T10Y2Y', 'VIXCLS']:
  if adfuller(econ[col])[1] > 0.05 and col != 'GDP':
    econ[col+'_diff'] = econ[col].diff() # create first diff
  econ[col]=econ[col].shift(1) # avoid overlap
econ.dropna(inplace=True)

if (adfuller(econ['gdp_growth'])[1]) > 0.05: # not possible at all, just in case
  econ['gdp_growth_diff'] = econ['gdp_growth'].diff()
econ.dropna(inplace=True)

# print(econ.head())

def add_prefix(df, prefix): # use this function so everytime I run this part, it will not add prefix everytime
    df.columns = [prefix + col if not col.startswith(prefix) else col for col in df.columns]
    return df

cre = add_prefix(cre, 'cre_')
card = add_prefix(card, 'card_')

df = pd.merge(cre,card,left_index=True,right_index=True)
df = pd.merge(df,econ,left_index=True,right_index=True,how='inner') # The pct_change starts from 2001-06-30, so use the inner to discard other rows

df = df[['cre_loans', 'cre_chargeoffs', 'cre_pct', 'cre_pct_change', \
         'card_loans', 'card_chargeoffs', 'card_pct', 'card_pct_change', \
         'UNRATE', 'DCOILBRENTEU','DCOILBRENTEU_diff', 'GDP', 'gdp_growth', 'T10Y2Y', 'VIXCLS']]
dfstat = df[['card_pct','cre_pct_change','card_pct_change','UNRATE', 'DCOILBRENTEU_diff', 'gdp_growth', 'T10Y2Y', 'VIXCLS']] # to conduct an AR model, we need stationary data


## AR1 using OLS

We use a function `myar1` to conduct this model, and it is more readible. Because we want to use loc, so first create a copy of it, and clean them after taking the lag for ar1.

The only thing we want from this model is just the r-square, so we only kept this part of the model, and then store them in a dictionary for calling.

Using sort is quite easy and quick way to find the top model(s), I used to keep the top three, but to fulfill the requirements, I kept the top one. If we actually kept 3 of them, we may find out that for both cre and card charge off data, same factors can interpret the best.

If the dependent variables are `pct_change`, it is clear that the r-square will be much smaller. The professor said it is some what a feature of AR1 model, which is interesting and reasonable. If we try to use the `pct` itself, the r-square will be larger because the lag as a factor will explain most of it.

In [19]:
import itertools
import statsmodels.api as sm

# factors are all stationary and 1Q lagged to avoid overlap
explanatory_vars = ['UNRATE', 'DCOILBRENTEU_diff', 'gdp_growth', 'T10Y2Y', 'VIXCLS']
factor_combinations = list(itertools.combinations(explanatory_vars, 3))

# ar1 is creating 1Q lag and use it as independent var
def myar1(dependent_var, explanatory_vars, df):
    df = df.copy()
    df.loc[:, dependent_var + '_lag1'] = df[dependent_var].shift(1)
    dfc = df.dropna()
    X = dfc[[dependent_var + '_lag1'] + list(explanatory_vars)].dropna()
    y = dfc[dependent_var]
    X = sm.add_constant(X)
    return sm.OLS(y, X).fit().rsquared

results = [{'combo': combo,
            'r2_cre': myar1('cre_pct_change', combo, dfstat),
            'r2_card': myar1('card_pct_change', combo, dfstat),
            'r2_card_ori':  myar1('card_pct', combo, dfstat)}
           for combo in factor_combinations]

top_cre_result = sorted(results, key=lambda x: x['r2_cre'], reverse=True)[:1]
top_card_result = sorted(results, key=lambda x: x['r2_card'], reverse=True)[:1]
top_card_ori_result = sorted(results, key=lambda x: x['r2_card_ori'], reverse=True)[:1]

print("Top model for 'cre_pct_change' based on R²:")
print(f"Factors: {top_cre_result[0]['combo']}, R²: {top_cre_result[0]['r2_cre']:.3f}")
print("\nTop model for 'card_pct_change' based on R²:")
print(f"Factors: {top_card_result[0]['combo']}, R²: {top_card_result[0]['r2_card']:.3f}")
print("\nTop model for 'card_pct' based on R²:")
print(f"Factors: {top_card_ori_result[0]['combo']}, R²: {top_card_ori_result[0]['r2_card_ori']:.3f}")
_=0

Top model for 'cre_pct_change' based on R²:
Factors: ('DCOILBRENTEU_diff', 'T10Y2Y', 'VIXCLS'), R²: 0.410

Top model for 'card_pct_change' based on R²:
Factors: ('UNRATE', 'DCOILBRENTEU_diff', 'gdp_growth'), R²: 0.206

Top model for 'card_pct' based on R²:
Factors: ('DCOILBRENTEU_diff', 'gdp_growth', 'T10Y2Y'), R²: 0.874


### Some other function we tried

Usually, I think we may use MLE instead of OLS to calculate ar models. So, I tried it as well. For using MLE, we won't have r-squared to explain "how much the factors can explain the dependent var", yet it can assess the likelihood, which is not that intuitive from my perspective.

In [17]:
dfstat.index.freq = 'QE'

import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
import itertools

def arima_with_factors(dependent_var, factor_combination, df):
    X = df[list(factor_combination)]
    model = ARIMA(df[dependent_var], exog=X, order=(1, 0, 0))
    result = model.fit()
    return {'combo': factor_combination,
            'loglikelihood': result.llf,
            'aic': result.aic,
            'bic': result.bic}

explanatory_vars = ['UNRATE', 'DCOILBRENTEU_diff', 'gdp_growth', 'T10Y2Y', 'VIXCLS']
factor_combinations = list(itertools.combinations(explanatory_vars, 3))
results_cre = []
results_card = []

for combo in factor_combinations:
    cre_result = arima_with_factors('cre_pct_change', combo, dfstat)
    card_result = arima_with_factors('card_pct_change', combo, dfstat)
    results_cre.append(cre_result)
    results_card.append(card_result)

best_cre_loglikelihood = sorted(results_cre, key=lambda x: x['loglikelihood'], reverse=True)[0]
best_cre_aic = sorted(results_cre, key=lambda x: x['aic'])[0]
best_cre_bic = sorted(results_cre, key=lambda x: x['bic'])[0]
best_card_loglikelihood = sorted(results_card, key=lambda x: x['loglikelihood'], reverse=True)[0]
best_card_aic = sorted(results_card, key=lambda x: x['aic'])[0]
best_card_bic = sorted(results_card, key=lambda x: x['bic'])[0]

print("Best model for 'cre_pct_change' based on log-likelihood, AIC and BIC:")
print(f"Factors: {best_cre_loglikelihood['combo']}, Log-likelihood: {best_cre_loglikelihood['loglikelihood']:.3f}\
AIC: {best_cre_aic['aic']:.3f}, BIC: {best_cre_bic['bic']:.3f}")
print("\nBest model for 'card_pct_change' based on log-likelihood, AIC and BIC:")
print(f"Factors: {best_card_loglikelihood['combo']}, Log-likelihood: {best_card_loglikelihood['loglikelihood']:.3f}\
AIC: {best_card_aic['aic']:.3f}, BIC: {best_card_bic['bic']:.3f}")
print("Two methods, using OLS R-square or using loglikihood, AIC and BIC shows the same result.")

/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Best model for 'cre_pct_change' based on log-likelihood, AIC and BIC:
Factors: ('DCOILBRENTEU_diff', 'T10Y2Y', 'VIXCLS'), Log-likelihood: 118.927AIC: -225.854, BIC: -211.949

Best model for 'card_pct_change' based on log-likelihood, AIC and BIC:
Factors: ('UNRATE', 'DCOILBRENTEU_diff', 'gdp_growth'), Log-likelihood: 28.856AIC: -45.712, BIC: -31.808
Two methods, using OLS R-square or using loglikihood, AIC and BIC shows the same result.


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [14]:
pip install pmdarima # This is a package that can find the optimistic ARIMA model automatically, learnt from the apply coding for risk management seminar

In [13]:
# Using other models may have better result.
from pmdarima import auto_arima

def auto_arima_with_factors(dependent_var, factor_combination, df):
    X = df[list(factor_combination)]
    model = auto_arima(df[dependent_var], exogenous=X, seasonal=False, stepwise=True, trace=False)
    arima_model = model.arima_res_
    return {'combo': factor_combination,
            'order': model.order,
            'loglikelihood': arima_model.llf,
            'aic': arima_model.aic,
            'bic': arima_model.bic}

results_cre, results_card = [], []

for combo in factor_combinations:
    results_cre.append(auto_arima_with_factors('cre_pct_change', combo, dfstat))
    results_card.append(auto_arima_with_factors('card_pct_change', combo, dfstat))
best_cre_ll = sorted(results_cre, key=lambda x: x['loglikelihood'], reverse=True)[0]
best_card_ll = sorted(results_card, key=lambda x: x['loglikelihood'], reverse=True)[0]
print(f"Best model for 'cre_pct_change' based on log-likelihood:\nFactors: {best_cre_ll['combo']}, ARIMA Order: {best_cre_ll['order']}, Log-Likelihood: {best_cre_ll['loglikelihood']:.3f}, AIC: {best_cre_ll['aic']:.3f}, BIC: {best_cre_ll['bic']:.3f}")
print(f"\nBest model for 'card_pct_change' based on log-likelihood:\nFactors: {best_card_ll['combo']}, ARIMA Order: {best_card_ll['order']}, Log-Likelihood: {best_card_ll['loglikelihood']:.3f}, AIC: {best_card_ll['aic']:.3f}, BIC: {best_card_ll['bic']:.3f}")
print("For cre information, we have higher log-likelihood, but not for the card data.")

Best model for 'cre_pct_change' based on log-likelihood:
Factors: ('UNRATE', 'DCOILBRENTEU_diff', 'gdp_growth'), ARIMA Order: (1, 0, 1), Log-Likelihood: 116.355, AIC: -226.709, BIC: -219.757

Best model for 'card_pct_change' based on log-likelihood:
Factors: ('UNRATE', 'DCOILBRENTEU_diff', 'gdp_growth'), ARIMA Order: (0, 0, 0), Log-Likelihood: 20.410, AIC: -38.819, BIC: -36.502
For cre information, we have higher log-likelihood, but not for the card data.


## Step 3

Key variables that I think could impact charge-off rates include interest rates, such as the Federal Funds Rate, the housing price index (HPI), consumer credit growth, the debt-to-income ratio (DTI), and initial jobless claims.

-	Interest rates or consumer price index (CPI) fluctuations can significantly affect consumers’ ability to service their credit card debts.
-	The housing price index is a critical factor influencing the probability of charge-offs in commercial real estate (CRE) loans.
-	For banks, when facing financial strain, they may resort to charge-offs to reduce their exposure to risky assets or to improve liquidity management.

To produce accurate forecasts (for this month) using these models, firstly I need factor data of the last month, including the oil price, gdp and unemployment rate. Aside from the model, I would also need reliable macroeconomic forecasts, insights into potential monetary and fiscal policy changes, and industry-specific trends, such as CRE vacancy rates or shifts in consumer behavior for credit cards. These inputs would significantly enhance the accuracy and reliability of future predictions.

Our group got 14 points last time and did not got a comment, so this time we are adding more text in our assignment. Hope this time it will help! :)

In [ ]:
print(0^_^0) # this can be run lol